In [49]:
##########################
## implement a3 in squin ##
##########################
from bloqade import squin
from bloqade.types import Qubit
from kirin.dialects import ilist
from bloqade.cirq_utils import load_circuit
from bloqade.cirq_utils.emit import emit_circuit
from bloqade.cirq_utils import noise
import bloqade.stim
import numpy as np
from math import pi
from typing import Literal


# ---------------------------------------------------------------------------
# Steane [[7,1,3]] Encoding Functions
# ---------------------------------------------------------------------------
@squin.kernel
def steane_encode_zero_on(q: ilist.IList[Qubit, Literal[7]]):
    """Encode |0⟩^7 → |0⟩_L on the 7-qubit register q (Steane [[7,1,3]])."""
    squin.h(q[0])
    squin.h(q[1])
    squin.h(q[3])
    squin.cx(q[0], q[4])
    squin.cx(q[1], q[2])
    squin.cx(q[3], q[5])
    squin.cx(q[0], q[6])
    squin.cx(q[3], q[4])
    squin.cx(q[1], q[5])
    squin.cx(q[0], q[2])
    squin.cx(q[5], q[6])


@squin.kernel
def steane_encode_plus_on(q: ilist.IList[Qubit, Literal[7]]):
    """Encode 7 qubits as |+⟩_L: first |0⟩_L then transversal H."""
    steane_encode_zero_on(q)
    for i in range(7):
        squin.h(q[i])


# ---------------------------------------------------------------------------
# A3 Circuit: Fault-tolerant Steane syndrome extraction
# ---------------------------------------------------------------------------
@squin.kernel
def a3_circuit():
    """A3 circuit implementation in squin."""
    q = squin.qalloc(21)
    
    steane_encode_zero_on(q[0:7])
    # Qubits 0-6: logical data
    # Qubits 8-14 + 15-21: ancilla block

    # Prepare ancilla as |+⟩_L for first half (X-stabilizer syndrome)
    steane_encode_plus_on(q[7:14])

    # CNOTs: data → ancilla (X-syndrome extraction)
    for i in range(7):
        squin.cx(q[i], q[i+7])

    # Reset ancilla qubits 8-14
    #for i in range(8, 15):
    #    squin.reset(q[i])
    
    # Prepare ancilla as |0⟩_L for second half (Z-stabilizer syndrome)
    steane_encode_zero_on(q[14:21])

    # CNOTs: ancilla → data (Z-syndrome extraction)
    for i in range(7):
        squin.cx(q[i+14], q[i])

    # Transversal H on ancilla
    for i in range(7):
        squin.h(q[i+14])
    
    # Measure ancilla qubits 7-21 (Syndrome bits)
    for i in range(7, 21):
        squin.measure(q[i])

In [64]:

# ---------------------------------------------------------------------------
# Execute and sample
# ---------------------------------------------------------------------------

loops = 1
shots_per_loop = 1000
all_samples = []

# Noise channel
noise_model = noise.GeminiOneZoneNoiseModel(scaling_factor=1)

for i in range(loops):
    # Generate circuit
    cirq_enc = emit_circuit(a3_circuit)
    
    # Apply noise to this circuit
    noisy_cirq_enc = noise.transform_circuit(cirq_enc, model=noise_model)
    
    # Convert to stim circuit and sample
    squin_enc = load_circuit(noisy_cirq_enc)
    stim_enc = bloqade.stim.Circuit(squin_enc)
    sampler = stim_enc.compile_sampler()
    
    # Sample this noisy circuit
    samples = sampler.sample(shots=shots_per_loop)
    all_samples.extend(samples)

samples_enc = np.array(all_samples)

# Show diagram from last circuit
fig = stim_enc.diagram()

print("MSD/Steane encoding A3 circuit: 15 qubits (7 data + 7 ancilla + 1 unused)")
print(f"Total samples collected: {len(samples_enc)} ({loops} loops × {shots_per_loop} shots)")
print("Sample shape:", samples_enc.shape)
print("First 5 shots:\n", samples_enc[:5])
print(f"Error rate: {np.count_nonzero(samples_enc)/len(samples_enc):.4f}")
fig



MSD/Steane encoding A3 circuit: 15 qubits (7 data + 7 ancilla + 1 unused)
Total samples collected: 1000 (1 loops × 1000 shots)
Sample shape: (1000, 14)
First 5 shots:
 [[ True  True False  True False  True False  True  True  True  True False
   True  True]
 [False False False False False False False  True  True False  True False
  False  True]
 [ True False False  True  True False False False False False  True  True
   True  True]
 [False False False  True  True  True  True False False False  True False
  False False]
 [False  True False False  True False  True  True  True False False  True
  False False]]
Error rate: 6.9380


q0: -S_DAG-SQRT_X_DAG-S------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------PAULI_CHANNEL_1(0.0004102,0.0004102,0.0004112)-PAULI_CHANNEL_1(0.000806,0.000806,0.002458)----@--------------------------------------------------PAULI_CHANNEL_2[0](0.00022747,0.00022747,0.00151278,0.00022747,0.00014286,0.00014286,0.00014308,0.00022747,0.00014286,0.00014286,0.00014308,0.00151278,0.00014308,0.00014308,0.00142814)-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------PAULI_CHANNEL_1(0.000806,0.000806,0.002458)----------------------------------------------------------------------------------------------------------------------------------------------Z----------------------------------------------------------------------------------------------------------------------------------------------------------------------------PAULI_CHANNEL_1(0.0004102,0.0004102,0.0004112)-PAULI_CHANNEL_1(0.000806,0.000806,0.002458)---------------------------------------------------@---------PAULI_CHANNEL_2[0](0.00022747,0.00022747,0.00151278,0.00022747,0.00014286,0.00014286,0.00014308,0.00022747,0.00014286,0.00014286,0.00014308,0.00151278,0.00014308,0.00014308,0.00142814)---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------PAULI_CHANNEL_1(0.000806,0.000806,0.002458)----------------------------------------------------------------------------------------------------------------------------------------------Z--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------PAULI_CHANNEL_1(0.0004102,0.0004102,0.0004112)-PAULI_CHANNEL_1(0.000806,0.000806,0.002458)---------------------------------------------------@-PAULI_CHANNEL_2[0](0.00022747,0.00022747,0.00151278,0.00022747,0.00014286,0.00014286,0.00014308,0.00022747,0.00014286,0.00014286,0.00014308,0.00151278,0.00014308,0.00014308,0.00142814)-PAULI_CHANNEL_1(0.000806,0.000806,0.002458)----Z-----------------------------------------------------------------------------------------------------------------------------------------------------------------PAULI_CHANNEL_1(0.0004102,0.0004102,0.0004112)-PAULI_CHANNEL_1(0.0003066,0.0003066,0.0004639)--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------PAULI_CHANNEL_1(0.0005149,0.0005149,0.002185)------------------------------------------------------------------------------------------------

In [65]:
synd_dict = {1:[1,3,5,7],2:[4,5,6,7],3:[2,3,6,7]}

In [66]:
def to_parity(li):
    li_new = [-2*el+1 for el in li]
    return li_new

li = [False, True, False]

In [67]:
def find_syndrome(li, idx):
    li = to_parity(li)
    mult = 1
    for ind in idx:
        mult *= li[ind-1]
    return True if mult == 1 else False


In [68]:
def findZSyndrome(sample,idx):
    sample_z = sample[0:7]
    return find_syndrome(sample_z,idx)

def findXSyndrome(sample,idx):
    sample_x = sample[7:14]
    return find_syndrome(sample_x,idx)

In [69]:
def compute_syndromes(sample,ind_dict):
    x_synds = []
    z_synds = []
    for key in list(ind_dict.keys()):
        idx = ind_dict[key]
        x_synds.append(findXSyndrome(sample,idx))
        z_synds.append(findZSyndrome(sample,idx))
    return x_synds,z_synds

In [70]:
def find_good_rate(samples,ind_dict):
    tot_samples = len(samples)
    good_samples = 0
    for sample in samples:
        synd = compute_syndromes(sample,ind_dict)
        will_add = 1 if np.count_nonzero(synd) == 6 else 0
        good_samples+=will_add
    return (good_samples)/len(samples)

In [71]:
for one_loop in samples_enc:
    print("Sample Start")
    for sample in one_loop:
        print(compute_syndromes(sample, synd_dict))
    print("Sample End")


Sample Start


IndexError: invalid index to scalar variable.

In [72]:
print(find_good_rate(samples_enc,synd_dict))

0.468
